In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,051 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,202 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('sales')
spark.sql(""" SELECT * FROM sales""").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT ROUND(avg(price),2) AS avg_price, date_built 
FROM sales WHERE bedrooms = 4 
group by date_built order by date_built ASC """).show()



+---------+----------+
|avg_price|date_built|
+---------+----------+
|296800.75|      2010|
| 302141.9|      2011|
|298233.42|      2012|
|299999.39|      2013|
|299073.89|      2014|
|307908.86|      2015|
|296050.24|      2016|
|296576.69|      2017|
+---------+----------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT ROUND(avg(price),2) AS avg_price, date_built 
FROM sales where bedrooms = 3  AND bathrooms = 3 
group by date_built order by date_built  ASC """).show()


+---------+----------+
|avg_price|date_built|
+---------+----------+
|292859.62|      2010|
|291117.47|      2011|
|293683.19|      2012|
|295962.27|      2013|
|290852.27|      2014|
| 288770.3|      2015|
|290555.07|      2016|
|292676.79|      2017|
+---------+----------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
SELECT ROUND(avg(price),2) AS avg_price, date_built 
FROM sales 
WHERE bedrooms = 3  AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 group by date_built order by date_built  ASC """).show()




+---------+----------+
|avg_price|date_built|
+---------+----------+
|285010.22|      2010|
|276553.81|      2011|
|307539.97|      2012|
|303676.79|      2013|
|298264.72|      2014|
|297609.97|      2015|
| 293965.1|      2016|
|280317.58|      2017|
+---------+----------+



In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT ROUND(avg(view),2) AS view 
FROM sales where price >= 350000 """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----+
| view|
+-----+
|32.26|
+-----+

--- 0.3221323490142822 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales.
start_time = time.time()

# Using spark.sql, write a query to calculate the average price for a four-bedroom house sold in each year, rounded to two decimal places.
spark.sql("""CACHE TABLE sales""")

# Print out the runtime.
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.19986438751220703 seconds ---


In [22]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [25]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

filtered_data = spark.sql("""SELECT ROUND(avg(view),2) AS view from sales where price >= 350000 """).show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----+
| view|
+-----+
|32.26|
+-----+

--- 0.146165132522583 seconds ---


In [26]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.parquet('sales', mode = 'overwrite')
p_df=spark.read.parquet('sales')
p_df.createOrReplaceTempView('p_sales')
p_df.write.partitionBy('date_built').mode('overwrite').parquet('parquet_sales')

In [27]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('parquet_sales')
p_df_p.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [28]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('final_sales')

In [30]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT ROUND(avg(view),2) AS view 
FROM final_sales 
WHERE price >= 350000 """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+-----+
| view|
+-----+
|32.26|
+-----+

--- 0.42122507095336914 seconds ---


In [31]:
# 14. Uncache the home_sales temporary table.
spark.sql("""UNCACHE TABLE sales""")

DataFrame[]

In [32]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('sales')


False